In [1]:
import os
import sys
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
import utils.downloading
from utils.progress.log_progress import log_progress
from utils.files.file_helper import ensure_directory, get_all_files_from_subfolders
import utils.configuration
import numpy as np
import pandas as pd

/Users/bulhakovdmytro/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
config = utils.configuration.Configuration()

In [4]:
HISTORICAL_DATA_FOLDER = config.config['DEFAULT']['FLIGHT_DATA']
WEATHER_DATA_FOLDER = config.config['DEFAULT']['WEATHER_DATA']
WEATHER_API_KEY = config.config['DEFAULT']['NOAA_WEATHER_API_KEY']
WEATHER_CITY_CODES = config.config['DEFAULT']['WEATHER_CITY_CODES']
YEARS_TO_DOWNLOAD = config.config['DEFAULT']['WORKING_YEARS']

In [5]:
def download_weather_data(downloader, city_list, country, year_list, save_folder):
    logging.info('Starting to download weather data')
    for year in year_list:
        download_yearly_weather_data(downloader, city_list, country, year, save_folder)

In [6]:
def download_yearly_weather_data(downloader, city_list, country, year, save_folder):
    start_date = '{}-01-01'.format(year)
    end_date = '{}-01-01'.format(year + 1)
    
    logging.info('Requesting data for {}'.format(year))
    
    error_city_list = []
    
    for city in city_list:
        try:
            logging.info('Requesting data for city: {}'.format(city))
            city_weather = weather_downloader.get_weather_for_city_by_name(city, country, start_date, end_date)
            logging.info('Saving data for city: {}'.format(city))
            save_folder_name = get_folder_for_year(save_folder, year)
            ensure_directory(save_folder_name)

            city_weather.to_csv('{}/{}.csv'.format(save_folder_name, city), index=False)
        except Exception as e:
            logging.info('No weather found for city {}'.format(city))
            error_city_list.append(city)

    logging.info('Error cities: {}'.format(error_city_list))

In [7]:
def get_folder_for_year(folder_to_save, year):
    return '{}/{}'.format(folder_to_save, year)

In [8]:
def get_existing_airports_names(historical_data_folder):
    historical_files_list = get_all_files_from_subfolders(HISTORICAL_DATA_FOLDER)
    
    name_list = set()
    
    for file in historical_files_list:
        file_data = pd.read_csv(data_file, 
                           dtype={'QUARTER': int, 'FL_NUM': str,
                                 'OP_UNIQUE_CARRIER': str, 'TAIL_NUM': str,
                                 'ORIGIN_AIRPORT_ID': int, 'ORIGIN_AIRPORT_SEQ_ID': int,
                                 'ORIGIN_AIRPORT_SEQ_ID': int, 'ORIGIN_CITY_MARKET_ID': int,
                                 'ORIGIN': str, 'ORIGIN_CITY_NAME': str, 
                                 'DEST_AIRPORT_ID': int, 'DEST_AIRPORT_SEQ_ID': int,
                                 'DEST_CITY_MARKET_ID': int, 'DEST': str,
                                 'DEST_CITY_NAME': str, 'CRS_DEP_TIME': int,
                                 'DEP_DELAY': float, 'ARR_DELAY': float,
                                 'CANCELLED': int, 'CANCELLATION_CODE': str,
                                 'DIVERTED': int, 'CRS_ELAPSED_TIME': str,
                                 'CARRIER_DELAY': float, 'WEATHER_DELAY': float,
                                 'NAS_DELAY': float, 'SECURITY_DELAY': float,
                                 'LATE_AIRCRAFT_DELAY': float
                                 }, 
                           parse_dates=['FL_DATE'])
        unique_city_names = np.unique(file_data['ORIGIN_CITY_NAME'])        
        name_list.update(unique_city_names)
        
    return list(name_list)

In [9]:
weather_downloader = utils.downloading.NOAAWeatherDownloader(WEATHER_API_KEY, WEATHER_CITY_CODES)

In [10]:
city_names = get_existing_airports_names(HISTORICAL_DATA_FOLDER)

/Users/bulhakovdmytro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
download_weather_data(weather_downloader, city_names, 'US', YEARS_TO_DOWNLOAD, WEATHER_DATA_FOLDER)

INFO:root:Starting to download weather data
INFO:root:Requesting data for 2017
INFO:root:Requesting data for city: Grand Rapids, MI
INFO:root:Saving data for city: Grand Rapids, MI
INFO:root:Requesting data for city: Fort Smith, AR
INFO:root:Saving data for city: Fort Smith, AR
INFO:root:Requesting data for city: Lanai, HI
ERROR:root:Could not find city with the specified name: Lanai, HI
INFO:root:No weather found for city Lanai, HI
INFO:root:Requesting data for city: Aspen, CO
ERROR:root:Could not find city with the specified name: Aspen, CO
INFO:root:No weather found for city Aspen, CO
INFO:root:Requesting data for city: Columbia, MO
INFO:root:Saving data for city: Columbia, MO
INFO:root:Requesting data for city: South Bend, IN
INFO:root:Saving data for city: South Bend, IN
INFO:root:Requesting data for city: Fayetteville, NC
INFO:root:Saving data for city: Fayetteville, NC
INFO:root:Requesting data for city: North Bend/Coos Bay, OR
ERROR:root:Could not find city with the specified n

INFO:root:Requesting data for city: Los Angeles, CA
INFO:root:Saving data for city: Los Angeles, CA
INFO:root:Requesting data for city: Unalaska, AK
ERROR:root:Could not find city with the specified name: Unalaska, AK
INFO:root:No weather found for city Unalaska, AK
INFO:root:Requesting data for city: Hyannis, MA
ERROR:root:Could not find city with the specified name: Hyannis, MA
INFO:root:No weather found for city Hyannis, MA
INFO:root:Requesting data for city: Wrangell, AK
ERROR:root:Could not find city with the specified name: Wrangell, AK
INFO:root:No weather found for city Wrangell, AK
INFO:root:Requesting data for city: Santa Maria, CA
INFO:root:Saving data for city: Santa Maria, CA
INFO:root:Requesting data for city: Gunnison, CO
ERROR:root:Could not find city with the specified name: Gunnison, CO
INFO:root:No weather found for city Gunnison, CO
INFO:root:Requesting data for city: Worcester, MA
INFO:root:Saving data for city: Worcester, MA
INFO:root:Requesting data for city: Anc

ERROR:root:Could not find city with the specified name: Latrobe, PA
INFO:root:No weather found for city Latrobe, PA
INFO:root:Requesting data for city: Panama City, FL
INFO:root:Saving data for city: Panama City, FL
INFO:root:Requesting data for city: San Jose, CA
INFO:root:Saving data for city: San Jose, CA
INFO:root:Requesting data for city: Corpus Christi, TX
INFO:root:Saving data for city: Corpus Christi, TX
INFO:root:Requesting data for city: Gillette, WY
INFO:root:Saving data for city: Gillette, WY
INFO:root:Requesting data for city: Tampa, FL
INFO:root:Saving data for city: Tampa, FL
INFO:root:Requesting data for city: Lansing, MI
INFO:root:Saving data for city: Lansing, MI
INFO:root:Requesting data for city: Hartford, CT
INFO:root:Saving data for city: Hartford, CT
INFO:root:Requesting data for city: Charlottesville, VA
INFO:root:Saving data for city: Charlottesville, VA
INFO:root:Requesting data for city: Grand Junction, CO
INFO:root:Saving data for city: Grand Junction, CO
IN

INFO:root:Requesting data for city: Ketchikan, AK
ERROR:root:Could not find city with the specified name: Ketchikan, AK
INFO:root:No weather found for city Ketchikan, AK
INFO:root:Requesting data for city: Springfield, MO
INFO:root:Saving data for city: Springfield, MO
INFO:root:Requesting data for city: Hagerstown, MD
INFO:root:Saving data for city: Hagerstown, MD
INFO:root:Requesting data for city: CONCORD, NC
ERROR:root:Could not find city with the specified name: CONCORD, NC
INFO:root:No weather found for city CONCORD, NC
INFO:root:Requesting data for city: Ponce, PR
ERROR:root:Could not find city with the specified name: Ponce, PR
INFO:root:No weather found for city Ponce, PR
INFO:root:Requesting data for city: Houston, TX
INFO:root:No weather found for city Houston, TX
INFO:root:Requesting data for city: Idaho Falls, ID
INFO:root:Saving data for city: Idaho Falls, ID
INFO:root:Requesting data for city: St. George, UT
INFO:root:Saving data for city: St. George, UT
INFO:root:Reques

INFO:root:Saving data for city: Harrisburg, PA
INFO:root:Requesting data for city: Aguadilla, PR
ERROR:root:Could not find city with the specified name: Aguadilla, PR
INFO:root:No weather found for city Aguadilla, PR
INFO:root:Requesting data for city: Wichita, KS
INFO:root:Saving data for city: Wichita, KS
INFO:root:Requesting data for city: Kearney, NE
INFO:root:Saving data for city: Kearney, NE
INFO:root:Requesting data for city: Dickinson, ND
INFO:root:Saving data for city: Dickinson, ND
INFO:root:Requesting data for city: Lewiston, ID
INFO:root:Saving data for city: Lewiston, ID
INFO:root:Requesting data for city: Roanoke, VA
INFO:root:Saving data for city: Roanoke, VA
INFO:root:Requesting data for city: Muskegon, MI
INFO:root:Saving data for city: Muskegon, MI
INFO:root:Requesting data for city: Melbourne, FL
INFO:root:Saving data for city: Melbourne, FL
INFO:root:Requesting data for city: Reno, NV
INFO:root:Saving data for city: Reno, NV
INFO:root:Requesting data for city: Mobil

ERROR:root:Could not find city with the specified name: King Salmon, AK
INFO:root:No weather found for city King Salmon, AK
INFO:root:Requesting data for city: Petersburg, AK
ERROR:root:Could not find city with the specified name: Petersburg, AK
INFO:root:No weather found for city Petersburg, AK
INFO:root:Requesting data for city: Wenatchee, WA
INFO:root:Saving data for city: Wenatchee, WA
INFO:root:Requesting data for city: Lawton/Fort Sill, OK
ERROR:root:Could not find city with the specified name: Lawton/Fort Sill, OK
INFO:root:No weather found for city Lawton/Fort Sill, OK
INFO:root:Requesting data for city: Pensacola, FL
INFO:root:Saving data for city: Pensacola, FL
INFO:root:Requesting data for city: Chicago, IL
INFO:root:Saving data for city: Chicago, IL
INFO:root:Requesting data for city: Pullman, WA
INFO:root:Saving data for city: Pullman, WA
INFO:root:Requesting data for city: Albany, NY
INFO:root:Saving data for city: Albany, NY
INFO:root:Requesting data for city: Newark, NJ